In [1]:
import oracledb
import pandas as pd
import random
import string
import requests
from datetime import datetime, timedelta

In [2]:
oracledb.init_oracle_client(lib_dir=r"D:\\instantclient_23_9")

conn = oracledb.connect(
    user="KMJ",          # 사용자명
    password="DBStudy2025!",      # 비밀번호
    dsn="dbstudy1205_medium" # 접속 정보 (SQL Developer와 동일)
)
cur = conn.cursor()

In [3]:
# 현재 시퀀스 값 반환하기
def return_sequence_value(sequence_name):
	query = f'select {sequence_name}.nextval from dual'
	sequence = pd.read_sql(query, conn)
	return sequence.iat[0, 0]

In [2]:
korea_person_name = []

for i in range(1,20):
	url = f'https://www.name-ranking.com/api/ranking?from=2008&to=2025&gender=&length=&p={i}'
	data = requests.get(url).json()
	for j in data['data']:
		korea_person_name += [j['name']]

first_name = list('김이박최정강조윤장임한오서신권황안송전홍유')

In [8]:
name_df = pd.DataFrame([random.choice(first_name) + random.choice(korea_person_name) for i in range(5000)]).reset_index(drop=True)
name_df = name_df.rename(columns={0:'한국인이름'})
name_df.to_csv('한국인이름.csv',index=False)

In [ ]:
# 한국 이름 반환
def return_korean_name():
	name_df = pd.read_csv('한국인이름.csv')
	return random.choice(name_df['한국인이름'])

In [6]:
# 랜덤으로 이메일 생성
def random_email():
    domains = ["gmail.com", "naver.com", "daum.net", "hotmail.com", "yahoo.com"]
    name_length = random.randint(6, 12)  # 아이디 길이
    name = ''.join(random.choices(string.ascii_lowercase + string.digits, k=name_length))
    domain = random.choice(domains)
    return f"{name}@{domain}"

In [7]:
# 랜덤으로 전화번호 생성
def random_phone_number():
    f = str(random.randint(1,9999))
    s = str(random.randint(1,9999))
    middle = '0' * (4-len(f)) + f
    last = '0' * (4-len(s)) + s
    phone_number = f'010-{middle}-{last}'
    return phone_number

In [8]:
# 랜덤으로 비밀번호 생성
def random_pwd():
    len_pwd = random.randint(12,16)  # 비밀번호 길이
    pwd = ''.join(random.choices(string.ascii_lowercase + string.digits + random.choice(string.punctuation), k=len_pwd))
    return pwd

In [9]:
# https://business.juso.go.kr/addrlink/attrbDBDwld/attrbDBDwldList.do?cPath=99MD&menu=%EB%8F%84%EB%A1%9C%EB%AA%85#this
address_list = []

with open("TN_SPRD_RDNM.txt", "r", encoding="euc-kr") as f:
	try:
		for line in f:
			doro = line.strip().split('|')
			bun = random.randint(1,200)
			bun2 = random.randint(1,200)
			address = ' '.join([doro[5], doro[6], doro[3], f"{str(bun)}-{str(bun2)}"])
			address_list.append(address)
	except:
		pass

pd.DataFrame(address_list, columns=['address']).to_csv('주소.csv',index=False)

In [10]:
def random_address():
	address = pd.read_csv('주소.csv')
	return random.choice(address['address'])

In [11]:
def random_birthday():
	return datetime.today() - timedelta(random.randint(5000,20000))

def random_gender():
    return random.choice(['F','M'])

In [12]:
query = 'select * from 회원'
db = pd.read_sql(query, conn)
db.columns

C:\Users\amysm\AppData\Local\Temp\ipykernel_5900\672279764.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  db = pd.read_sql(query, conn)


Index(['회원ID', '회원명', '전화번호', '주소', '상세주소', '회원등급', '포인트', '이메일', '비밀번호',
       '생년월일', '성별', '탈퇴여부', '탈퇴날짜'],
      dtype='object')

In [1]:
def generate_user(cols):
    try:
        data = [
            0,
            return_korean_name(),
            random_phone_number(),
            random_address(),
            '.',
            random.choices(
                ['Welcome', 'Family', 'Vip', 'Platinum'],
                weights=[0.7, 0.15, 0.1, 0.05],
                k=1
            )[0],
            random.choices(range(0, 10001, 50), k=1)[0],
            random_email(),
            random_pwd(),
            random_birthday(),
            random_gender(),
            None,
            None
        ]
        return dict(zip(cols, data))
    except Exception:
        return None

In [12]:
import pandas as pd
df = pd.read_csv('users.csv')

In [13]:
df.columns

Index(['회원ID', '회원명', '전화번호', '주소', '상세주소', '회원등급', '포인트', '이메일', '비밀번호',
       '생년월일', '성별', '탈퇴여부', '탈퇴날짜'],
      dtype='object')

In [14]:
df = df.drop_duplicates('전화번호')
df = df.drop_duplicates('이메일')
df = df.drop_duplicates('비밀번호')

In [15]:
df = df.reset_index(drop=True)
df['회원ID'] = list(range(1,df.shape[0]+1))

In [16]:
df.to_csv('데이터_회원테이블.csv',index=False)